In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import src.models.helpers as mh

import src.data.breathe_data as bd
import src.data.helpers as dh
from scipy.stats import norm
from scipy.stats import laplace

import itertools

In [3]:
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_conservative_smoothing_with_idx")

In [4]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,idx ecFEV1 (L),idx O2 saturation (%),idx ecFEF2575%ecFEV1,idx ecFEF25-75 % ecFEV1 (%)
0,101,2019-01-25,1.31,97,0.54,1.31,0.54,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,99.845492,41.221374,26,47,20,20
1,101,2019-01-26,1.31,98,0.57,1.31,0.57,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,100.874827,43.511450,26,48,21,21
2,101,2019-01-27,1.31,96,0.67,1.31,0.67,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,98.816157,51.145038,26,46,25,25
3,101,2019-01-28,1.30,96,0.69,1.30,0.69,Male,173.0,53,3.610061,97.150104,36.010470,36.010470,98.816157,53.076923,26,46,26,26
4,101,2019-01-29,1.28,98,0.60,1.28,0.60,Male,173.0,53,3.610061,97.150104,35.456463,35.456463,100.874827,46.875000,25,48,23,23


In [67]:
def get_day_to_day_ecFEV1(df_for_ID, n_days):
    df_for_ID = df_for_ID.sort_values(by="Date Recorded", ascending=True)
    df_for_ID["day_to_day_ecFEV1"] = df_for_ID["ecFEV1"].diff(periods=n_days)
    return df_for_ID

n_days=1
df_d2d = df.groupby("ID").apply(get_day_to_day_ecFEV1, n_days=n_days)
d2d = df_d2d["day_to_day_ecFEV1"].dropna().reset_index().drop(columns=["level_1"])

In [ ]:
# Plot histogram of 
fig = go.Figure()
xbins = dict(
    start=-0.505,
    end=0.5,
    size=0.01
)
fig.add_trace(go.Histogram(x=d2d["day_to_day_ecFEV1"], xbins=xbins, histnorm='probability'))

weight = 0.7
# Add gaussian with std as d2d
mu = d2d["day_to_day_ecFEV1"].mean()
std = d2d["day_to_day_ecFEV1"].std()
print(f"mu: {mu}, std: {std}")
x = np.linspace(-0.5, 0.5, 100)
y1 = norm.pdf(x, mu, std*1.1)
y1 = (1-weight) * y1 / y1.sum()
fig.add_trace(go.Scatter(x=x, y=y1, mode='lines', name="Gaussian", line=dict(color='red')))
# Add laplace
y2 = norm.pdf(x, mu, 0.05)
y2 = laplace.pdf(x, mu, 0.05)
y2 = weight * y2 / y2.sum()
fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', name="Laplace", line=dict(color='green')))

y = y1 + y2
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name="Combined", line=dict(color='blue')))

fig.update_xaxes(title_text="Day-to-day change in ecFEV1")
fig.update_layout(title=f"Change in ecFEV1 over {n_days} days")
fig.show()

mu: 0.0012513444802972522, std: 0.11730310179854178


In [64]:
# Plot this per individual
for id in d2d["ID"].unique():
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=d2d[d2d["ID"] == id]["day_to_day_ecFEV1"], xbins=xbins, histnorm='probability'))
    fig.update_xaxes(title_text="Day-to-day change in ecFEV1", range=[-0.5, 0.5])
    title = f"ID {id} - Change in ecFEV1 over {n_days} days"
    fig.update_layout(title=title, width=800, height=400)
    fig.write_image(f"{dh.get_path_to_main()}/PlotsBreathe/Interconnecting_ARs_entries/day_to_day_ecFEV1/{title}.pdf")